In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from utils import show_recipe

C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "model/finetuned_recipenlg_2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model and tokenizer loaded.")


prompt = (
    "<RECIPE_START>"
    "<INPUT_START>milk<NEXT_INPUT>cream<NEXT_INPUT>strawberry<NEXT_INPUT>melon<NEXT_INPUT>honey<INPUT_END>"
)

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=400,
    return_attention_mask=True,
)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=400,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True,
    eos_token_id=tokenizer.convert_tokens_to_ids("<RECIPE_END>"),
)

raw_recipe = tokenizer.decode(outputs[0])
show_recipe(raw_recipe)

Setting `pad_token_id` to `eos_token_id`:50266 for open-end generation.


Model and tokenizer loaded.
## Recipe Title: Melon-Jello Mold

Inputs (base ingredients):
   - milk
   - cream
   - strawberry
   - melon
   - honey

Ingredients:
   - 1 1/2 cups milk
   - 1 cup half-and-half cream
   - 10 ounces strawberry Jell-O gelatin dessert
   - 8 ounces cantaloupe melon, cubed
   - 2 tablespoons honey

Instructions:
   1. In a medium saucepan, combine milk and cream. Cook and stir over medium heat until mixture comes to a boil. Remove from heat; stir in gelatin until dissolved. Chill until slightly thickened, about 1 hour.
   2. Stir in melons and honey. Pour into a 6-cup mold coated with cooking spray. Refrigerate until set, at least 4 hours. Unmold onto a serving platter.
